# Dataset Generation Script

This script performs the following operations:
1. Extracts 5000 training questions and 100 testing questions for each domain (coding, math, trivia).
2. Saves the selected datasets.
3. Generates responses using 4 different teacher models via OpenRouter.
4. Applies templates in half of the cases.
5. Saves the generated data.

In [11]:
import os
import json
import random
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
# from tqdm.notebook import tqdm # Causing ImportError with IProgress
from tqdm import tqdm # Use standard tqdm instead
from openai import OpenAI

# --- Configuration ---

# Ensure you have your OpenRouter API key set in your environment
# os.environ["OPENROUTER_API_KEY"] = "sk-or-v1-..."
API_KEY = os.getenv("OPENROUTER_API_KEY")

if not API_KEY:
    raise ValueError("Please set OPENROUTER_API_KEY environment variable.")

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=API_KEY,
)

DOMAINS = ["trivia", "math", "general"]
TEACHER_MODELS = [
    "qwen/qwen-2.5-72b-instruct",
    "qwen/qwen-2.5-7b-instruct",
    "meta-llama/llama-3.1-70b-instruct",
    "meta-llama/llama-3.1-8b-instruct",
]
TRAIN_SIZE = 5000
TEST_SIZE = 100
SEED = 42
MAX_WORKERS = 5 # reduced per-config workers since we run configs in parallel
MAX_PARALLEL_CONFIGS = 4 # Number of configurations to run simultaneously


In [12]:
try:
    print("Testing OpenRouter connection...")
    completion = client.chat.completions.create(
        model="meta-llama/llama-3.1-8b-instruct",
        messages=[
            {"role": "user", "content": "Hello, world!"}
        ]
    )
    print("Connection successful!")
    print("Response:", completion.choices[0].message.content)
except Exception as e:
    print("Connection failed:", str(e))
    raise e

Testing OpenRouter connection...
Connection successful!
Response: Hello! It's nice to meet you! Is there something I can help you with or would you like to chat?


In [13]:
def load_and_select_data(domain, split, count):
    """
    Loads data for a domain/split.
    If 'selected' file exists, returns it.
    Otherwise, samples from 'raw' file, saves to 'selected', and returns it.
    """
    base_path = os.path.join("experiments", domain)
    selected_file = os.path.join(base_path, f"{split}_selected.jsonl")
    raw_file = os.path.join(base_path, f"{split}_raw.jsonl")

    # Check if selected file exists
    if os.path.exists(selected_file):
        print(f"Loading existing selected data for {domain}/{split}...")
        with open(selected_file, 'r', encoding='utf-8') as f:
            data = [json.loads(line) for line in f]
        # Verify size (warn if mismatch, but allow reuse)
        if len(data) != count:
            print(f"Warning: Existing file has {len(data)} items, expected {count}.")
        return data

    print(f"Sampling new data for {domain}/{split}...")
    # Load raw data
    data = []
    with open(raw_file, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line))

    # Sample
    random.seed(SEED)
    if len(data) > count:
        selected_data = random.sample(data, count)
    else:
        selected_data = data

    # Save selected
    with open(selected_file, 'w', encoding='utf-8') as f:
        for item in selected_data:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")

    return selected_data

def load_template(domain):
    path = os.path.join("experiments", domain, "template.txt")
    with open(path, 'r', encoding='utf-8') as f:
        return f.read().strip()

In [14]:
def generate_single_response(item, model, template_text, use_template):
    """
    Generates a response for a single item.
    Returns the enriched item with 'response', 'model', 'template_applied'.
    """
    question = item['question']

    if use_template and template_text:
        # Assume template has [QUESTION] placeholder based on file inspection
        prompt = template_text.replace("[QUESTION]", question)
    else:
        prompt = question

    # Retry logic
    max_retries = 3
    for attempt in range(max_retries):
        try:
            completion = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "user", "content": prompt}
                ]
            )
            response = completion.choices[0].message.content

            result = item.copy()
            result['response'] = response
            result['model'] = model
            result['template_applied'] = use_template
            result['prompt'] = prompt
            return result

        except Exception as e:
            if attempt == max_retries - 1:
                print(f"Failed to generate for prompt: {prompt[:50]}... Error: {e}")
                result = item.copy()
                result['error'] = str(e)
                return result
            time.sleep(2 ** attempt)  # Exponential backoff

In [15]:
def run_generation_process(domain, split, data, model, use_template):
    """
    Runs generation for a specific configuration and saves results.
    """
    template_text = load_template(domain)

    # Construct output filename
    # Format: experiments/{domain}/{split}_generated_{model_clean}_{template}.jsonl
    model_clean = model.replace("/", "_")
    template_str = "with_template" if use_template else "no_template"
    output_filename = f"{split}_generated_{model_clean}_{template_str}.jsonl"
    output_path = os.path.join("experiments", domain, output_filename)

    if os.path.exists(output_path):
        print(f"  [Skipping] {output_filename} already exists.")
        return

    print(f"  [Generating] {domain} | {split} | {model} | Template={use_template}")

    results = []
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = [
            executor.submit(generate_single_response, item, model, template_text, use_template)
            for item in data
        ]

        # Using tqdm for progress within the generation
        for future in tqdm(as_completed(futures), total=len(data), desc=f"    Progress", leave=False):
            results.append(future.result())

    # Save results
    with open(output_path, 'w', encoding='utf-8') as f:
        for res in results:
            f.write(json.dumps(res, ensure_ascii=False) + "\n")
    print(f"  [Saved] {output_filename}")

In [ ]:
# --- Main Execution Loop ---

# Pre-load all data serially to avoid race conditions
domain_data = {}
for domain in DOMAINS:
    print(f"Loading data for {domain}...")
    try:
        train_data = load_and_select_data(domain, "train", TRAIN_SIZE)
        test_data = load_and_select_data(domain, "test", TEST_SIZE)
        domain_data[domain] = {"train": train_data, "test": test_data}
    except Exception as e:
        print(f"Skipping domain {domain} due to error: {e}")

# Collect all tasks
tasks = []
for domain, datasets in domain_data.items():
    for model in TEACHER_MODELS:
        for use_template in [True, False]:
            for split_name in ["train", "test"]:
                tasks.append({
                    "domain": domain,
                    "split": split_name,
                    "data": datasets[split_name],
                    "model": model,
                    "use_template": use_template
                })

print(f"\nTotal configurations to run: {len(tasks)}")

def process_task(task):
    try:
        run_generation_process(
            domain=task["domain"],
            split=task["split"],
            data=task["data"],
            model=task["model"],
            use_template=task["use_template"]
        )
        return f"Success: {task['domain']} {task['model']}"
    except Exception as e:
        return f"Error in {task['domain']} {task['model']}: {e}"

print("Starting parallel execution...")
with ThreadPoolExecutor(max_workers=MAX_PARALLEL_CONFIGS) as executor:
    futures = [executor.submit(process_task, task) for task in tasks]

    for future in as_completed(futures):
        print(future.result())

print("\nAll generation tasks completed!")

Loading data for trivia...
Loading existing selected data for trivia/train...
Loading existing selected data for trivia/test...
Loading data for math...
Loading existing selected data for math/train...
Loading existing selected data for math/test...
Loading data for general...
Loading existing selected data for general/train...
Loading existing selected data for general/test...

Total configurations to run: 48
Starting parallel execution...
  [Skipping] test_generated_qwen_qwen-2.5-72b-instruct_with_template.jsonl already exists.
  [Generating] trivia | train | qwen/qwen-2.5-72b-instruct | Template=False
  [Skipping] train_generated_qwen_qwen-2.5-72b-instruct_with_template.jsonl already exists.
Success: trivia qwen/qwen-2.5-72b-instruct
Success: trivia qwen/qwen-2.5-72b-instruct
  [Skipping] test_generated_qwen_qwen-2.5-72b-instruct_no_template.jsonl already exists.
Success: trivia qwen/qwen-2.5-72b-instruct
  [Skipping] test_generated_qwen_qwen-2.5-7b-instruct_with_template.jsonl alre

    Progress:   0%|          | 0/5000 [00:00<?, ?it/s]


    Progress:   0%|          | 1/5000 [00:00<40:04,  2.08it/s]















    Progress:   0%|          | 2/5000 [00:03<2:43:00,  1.96s/it]





    Progress:   0%|          | 3/5000 [00:04<1:54:32,  1.38s/it]


















    Progress:   0%|          | 4/5000 [00:06<2:15:28,  1.63s/it]














    Progress:   0%|          | 6/5000 [00:08<1:42:35,  1.23s/it]






    Progress:   0%|          | 8/5000 [00:09<1:15:57,  1.10it/s]



    Progress:   0%|          | 9/5000 [00:09<1:06:24,  1.25it/s]














    Progress:   0%|          | 10/5000 [00:11<1:32:55,  1.12s/it]








    Progress:   0%|          | 11/5000 [00:13<1:36:01,  1.15s/it]

    Progress:   0%|          | 12/5000 [00:13<1:18:53,  1.05it/s]



    Progress:   0%|          | 14/5000 [00:14<1:03:16,  1.31it/s]
















    Progress:   0%|          | 15/5000 [00:17<1:41:09,  1.22s/it]

    Progress:   0%|          | 16/5000 [00:17<1:28:21,  

  [Saved] train_generated_meta-llama_llama-3.1-70b-instruct_with_template.jsonl
Success: trivia meta-llama/llama-3.1-70b-instruct
  [Generating] trivia | test | meta-llama/llama-3.1-70b-instruct | Template=False







    Progress:  25%|██▍       | 1239/5000 [14:33<1:21:36,  1.30s/it]













    Progress:  25%|██▍       | 1241/5000 [14:35<1:14:55,  1.20s/it]

    Progress:  25%|██▍       | 1243/5000 [14:36<46:36,  1.34it/s]












    Progress:  25%|██▍       | 1244/5000 [14:37<1:02:17,  1.00it/s]













    Progress:  25%|██▍       | 1245/5000 [14:39<1:22:35,  1.32s/it]








    Progress:  25%|██▍       | 1246/5000 [14:40<1:16:49,  1.23s/it]







    Progress:  25%|██▍       | 1247/5000 [14:42<1:15:36,  1.21s/it]





    Progress:  25%|██▍       | 1248/5000 [14:43<1:10:06,  1.12s/it]

    Progress:  25%|██▍       | 1249/5000 [14:43<59:03,  1.06it/s]  







    Progress:  25%|██▌       | 1251/5000 [14:44<49:25,  1.26it/s]











    Progress:  25%|██▌       | 1252/5000 [14:46<1:01:39,  1.01it/s]






    Progress:  25%|██▌       | 1253/5000 [14:47<1:04:15,  1.03s/it]



    Progress:  25%|██▌       | 1255/5000 [14:48<47:59,  1.30it/s]











    Progress:  25%|

  [Saved] test_generated_meta-llama_llama-3.1-70b-instruct_no_template.jsonl
Success: trivia meta-llama/llama-3.1-70b-instruct
  [Generating] trivia | train | meta-llama/llama-3.1-8b-instruct | Template=True


...

    Progress:   0%|          | 0/5000 [00:00<?, ?it/s]







    Progress:   0%|          | 6/5000 [00:06<1:13:12,  1.14it/s]





    Progress:   0%|          | 7/5000 [00:07<1:20:57,  1.03it/s]


    Progress:   0%|          | 8/5000 [00:09<1:32:06,  1.11s/it]







    Progress:   0%|          | 9/5000 [00:10<1:43:22,  1.24s/it]

    Progress:   0%|          | 11/5000 [00:12<1:23:50,  1.01s/it]




    Progress:   0%|          | 12/5000 [00:14<1:45:39,  1.27s/it]

    Progress:   0%|          | 13/5000 [00:14<1:20:37,  1.03it/s]

    Progress:   0%|          | 14/5000 [00:17<2:01:24,  1.46s/it]





    Progress:   0%|          | 16/5000 [00:19<1:31:27,  1.10s/it]







    Progress:   0%|          | 18/5000 [00:21<1:32:14,  1.11s/it]



    Progress:   0%|          | 20/5000 [00:22<1:09:16,  1.20it/s]


    Progress:   0%|          | 21/5000 [00:24<1:22:22,  1.01it/s]





    Progress:   0%|          | 22/5000 [00:27<2:10:33,  1.57s/it]





    Progress:   0%|          | 23/50

  [Saved] train_generated_meta-llama_llama-3.1-70b-instruct_no_template.jsonl
Success: trivia meta-llama/llama-3.1-70b-instruct
  [Generating] math | test | qwen/qwen-2.5-7b-instruct | Template=True




    Progress:   1%|▏         | 66/5000 [01:07<1:00:51,  1.35it/s]


    Progress:   1%|▏         | 67/5000 [01:09<1:16:24,  1.08it/s]

    Progress:   1%|▏         | 68/5000 [01:09<1:06:01,  1.24it/s]

    Progress:   1%|▏         | 69/5000 [01:10<58:21,  1.41it/s]  



    Progress:   1%|▏         | 70/5000 [01:10<52:41,  1.56it/s]



    Progress:   1%|▏         | 72/5000 [01:11<51:32,  1.59it/s]  



    Progress:   2%|▏         | 78/5000 [01:16<45:21,  1.81it/s]






    Progress:   2%|▏         | 79/5000 [01:17<59:33,  1.38it/s]


    Progress:   2%|▏         | 80/5000 [01:19<1:20:18,  1.02it/s]



    Progress:   2%|▏         | 81/5000 [01:20<1:32:12,  1.12s/it]

    Progress:   2%|▏         | 82/5000 [01:21<1:22:39,  1.01s/it]







    Progress:   2%|▏         | 86/5000 [01:24<53:35,  1.53it/s]  



    Progress:   2%|▏         | 89/5000 [01:26<51:41,  1.58it/s]  


    Progress:   2%|▏         | 92/5000 [01:28<51:58,  1.57it/s]  


    Progress:   2%|▏         | 94/5000 [0

  [Saved] test_generated_qwen_qwen-2.5-7b-instruct_with_template.jsonl
Success: math qwen/qwen-2.5-7b-instruct
  [Generating] math | train | qwen/qwen-2.5-7b-instruct | Template=False





    Progress:   3%|▎         | 165/5000 [02:36<1:09:36,  1.16it/s]

    Progress:   3%|▎         | 166/5000 [02:37<1:02:58,  1.28it/s]

    Progress:   3%|▎         | 168/5000 [02:39<1:06:30,  1.21it/s]




    Progress:   3%|▎         | 170/5000 [02:41<1:17:09,  1.04it/s]


    Progress:   3%|▎         | 172/5000 [02:43<1:14:19,  1.08it/s]

    Progress:   3%|▎         | 173/5000 [02:45<1:30:24,  1.12s/it]



    Progress:   3%|▎         | 174/5000 [02:45<1:17:38,  1.04it/s]


    Progress:   4%|▎         | 177/5000 [02:47<54:54,  1.46it/s]  

    Progress:   4%|▎         | 178/5000 [02:48<1:06:55,  1.20it/s]




    Progress:   4%|▎         | 180/5000 [02:51<1:21:41,  1.02s/it]


    Progress:   4%|▎         | 183/5000 [02:53<1:07:28,  1.19it/s]


    Progress:   4%|▎         | 186/5000 [02:56<1:04:00,  1.25it/s]

    Progress:   4%|▎         | 187/5000 [02:57<1:08:26,  1.17it/s]


    Progress:   4%|▍         | 192/5000 [03:00<50:17,  1.59it/s]


    Progress:   4%|▍         | 19